In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import datetime as dt
import random
import time
import itertools
import math
from graphviz import Digraph
import scipy.signal as sig
import timeit

In [3]:
almNum = 3
ocorr = 10
t_sample = 60
base_hour = dt.datetime(2018, 1,9,9,0,0)
occor_num = 0

#Settings alarms
a = np.zeros(1, dtype=int)
duration = 120 #120sec
hour_init = base_hour

b = np.zeros(1, dtype=int)
pAb = 1
delay_b = 120#seconds
duration_b =120

c = np.zeros(1, dtype=int)
pAc = 1
delay_c = 120#seconds
duration_c = 120

#Alarm Series Generation, A (cause), B(Effect). C(Effect)
while(occor_num < ocorr): 
    #A generation - begin
    srtd_hour = random.normalvariate(3, 1)
    srtd_hour_begin = hour_init + dt.timedelta(hours=srtd_hour)
    srtd_hour_end = srtd_hour_begin + dt.timedelta(seconds = duration)

    idx_init_a_occor = int(math.ceil((srtd_hour_begin - base_hour).total_seconds()/t_sample))
    idx_end_a_occor = int(math.ceil((srtd_hour_end - base_hour).total_seconds()/t_sample))
        
    if(idx_end_a_occor > a.size):
            a.resize(idx_end_a_occor)

    for i in np.arange(idx_init_a_occor, idx_end_a_occor + 1):
        a[i-1] = 1
    #A generation - end
         
    
    #B generation begin
    srtd_prob_b = random.uniform(0,1)  
    if srtd_prob_b <= pAb:
        srtd_hour_begin_b = srtd_hour_begin + dt.timedelta(seconds = delay_b)
        srtd_hour_end_b = srtd_hour_begin_b + dt.timedelta(seconds=duration_b)

        idx_init_b_occor = int(math.ceil((srtd_hour_begin_b - base_hour).total_seconds()/t_sample))
        idx_end_b_occor = int(math.ceil((srtd_hour_end_b - base_hour).total_seconds()/t_sample))

        if(idx_end_b_occor > b.size):
                b.resize(idx_end_b_occor)

        for j in np.arange(idx_init_b_occor, idx_end_b_occor +1):
            b[j-1] =  1  
    #B generation end

    #C generation begin
    srtd_prob_c = random.uniform(0,1)
    if srtd_prob_c <= pAc:
        srtd_hour_begin_c = srtd_hour_begin + dt.timedelta(seconds = delay_c)
        srtd_hour_end_c = srtd_hour_begin_c + dt.timedelta(seconds=duration_c)

        idx_init_c_occor = int(math.ceil((srtd_hour_begin_c - base_hour).total_seconds()/t_sample))
        idx_end_c_occor = int(math.ceil((srtd_hour_end_c - base_hour).total_seconds()/t_sample))
        
        if(idx_end_c_occor > c.size):
            c.resize(idx_end_c_occor)

        for j in np.arange(idx_init_c_occor, idx_end_c_occor +1):
            c[j-1] = 1    
    #C generation end
    
    hour_init = srtd_hour_begin
    occor_num = occor_num + 1

#Making series the same length
max_len = max(a, b, c, key=len).size
a = np.concatenate([a, np.zeros(max_len - a.size)])
b = np.concatenate([b, np.zeros(max_len - b.size)])
c = np.concatenate([c, np.zeros(max_len - c.size)])
sum(c)

30.0

In [4]:
def graph(df, limite, eng = 'dot'):
    g = Digraph(engine=eng)
    for k, row in enumerate(df.index):
        if any(df.iloc[k]>=limite) or any(df[row] >= limite):
            g.node(str(k),row, shape='oval', fontsize='10', width='0') 

    for j, col in enumerate(df.columns):
        for i, row in enumerate(df[col]):
            if row  >=limite:
                g.edge(str(i), str(j), label=str(np.round(row,3)),\
                       color="#000000{:02x}".format(int(255)))
                
                       #* row//max(df.max()))
    return g    

In [5]:
def plot_mat_corrs(figsize, annot, matrix):
    f, ax = plt.subplots(figsize=figsize)
    cmap = sns.diverging_palette(150, 275, s=80, l=55, as_cmap=True)
    sns.heatmap(matrix, cmap=cmap, center=0, annot=annot)
    plt.yticks(rotation=45)
    plt.xticks(rotation=45)
    plt.show()

In [6]:
#guess window_lag = ceil(delay_b_c/t_sample)

def interrelation(window_lag, matrix):
    #window_lag =  5#shoud be minor than size of the series

    print(matrix.shape)
    count_occor = 0;
    corr_mat = np.zeros([matrix.shape[1],matrix.shape[1]], dtype=float)
    alrms = {}
    for i in np.arange(0, matrix.shape[1]):
        alrms[i] = matrix[:,i]
    #print(alrms)
    for idx_caus, value in alrms.items():
        reduced_alarms = alrms.copy()
        del reduced_alarms[idx_caus]
        for idx_occor_caus, occor in enumerate(value):
            if occor == 1:
                if(idx_occor_caus+window_lag < len(value)):
                    count_occor = count_occor + 1
                    for idx_fct, alm_redu in reduced_alarms.items():
                        counted = False
                        for idx_occor_fct in np.arange(idx_occor_caus+1, idx_occor_caus+1+window_lag):
                            if alm_redu[idx_occor_fct] == 1 and  not counted:
                                counted = True
                                corr_mat[idx_caus][idx_fct] = corr_mat[idx_caus][idx_fct]+1
        if count_occor > 0: 
            corr_mat[idx_caus] = corr_mat[idx_caus]/count_occor
            count_occor = 0
    return corr_mat

In [7]:
def app_roll_mean(df, window):
    roll  = df.rolling(window).mean() 
    roll.dropna(inplace=True)
    roll = roll.round(decimals=0).copy()
    roll.reset_index(drop=True, inplace=True)
    return roll.copy()

In [8]:
def generate_df_valid_corrs(df, limit):
    df_valid = pd.DataFrame(np.zeros([df.shape[0], df.shape[1]], dtype=int), columns = df.columns, index= df.columns)
    for row in df.columns:
        for col in df.columns:
            if df[row][col] > limit:
                df_valid[row][col] = 1
                
    return df_valid 

In [5]:
#plot_mat_corrs(figsize=(30,20), annot=True, matrix=corr)

In [9]:
dist6_novo = pd.read_csv("dist6_3horas_sig/alm_seq.csv")
dist = dist6_novo[['xmeas%02d_low' % x for x in [1,2,3,8,9,21]] + ['xmeas%02d_high' % x for x in [1,2,3,8,9,21]]]

In [ ]:
def apply_mean(man_samp, df_dist):
    df_dist.drop('tout', axis=1, inplace=True)

    dist_sel = df_dist[['xmeas%02d_low' % x for x in [1,2,3,8,9,21]] + ['xmeas%02d_high' % x for x in [1,2,3,8,9,21]]]
    roll_1  = dist_sel.rolling(man_samp).mean() 
    roll_1.dropna(inplace=True)
    roll_1 = roll_1.round(decimals=0,).copy()
    roll_1.reset_index(drop=True, inplace=True)

    dist_first_diff = roll_1.diff(1)
    dist_first_diff.iloc[0] = roll_1.iloc[0]
    
    return dist_first_diff

In [37]:
cen_normal = "qualificacao/normal/inter/"
cen_media2 = "qualificacao/mean2/inter/"
cen_media5 = "qualificacao/mean5/inter/"

In [28]:
df_mean2 = app_roll_mean(dist,2)

In [30]:
df_mean5 = app_roll_mean(dist,5)

In [ ]:
# corr_move = interrelation(window, df_modified.as_matrix())

In [29]:
# t = np.mean(corr_move) + 3*np.std(corr_move)
# df_cors = pd.DataFrame(corr_move, columns= dist6_sel_vars.columns,      
#         index= dist6_sel_vars.columns)
# df_cors


# graph(df_cors, t, 'dot')

In [44]:
windows = ((1/(0.6))*np.arange(5,35,5)).astype(int)
windows

array([ 8, 16, 25, 33, 41, 50])

In [39]:
def inter_case(wind, df, path_cen):
    for w in wind:
        cor = interrelation(w, df.as_matrix())
        df_cors = pd.DataFrame(cor, columns= df.columns,index= df.columns)
        df_cors.to_csv(path_cen+str(w)+".csv")
        
        t = np.mean(cor) + 3*np.std(cor)
        valid_cor = generate_df_valid_corrs(df_cors,t)
        valid_cor.to_csv(path_cen+str(w)+".sel_corr.csv")     

In [40]:
inter_case(windows, dist, cen_normal)

(15351, 12)
(15351, 12)
(15351, 12)
(15351, 12)
(15351, 12)
(15351, 12)


In [42]:
inter_case(windows, df_mean2, cen_media2)

(15350, 12)
(15350, 12)
(15350, 12)
(15350, 12)
(15350, 12)
(15350, 12)


In [43]:
inter_case(windows, df_mean5, cen_media5)

(15347, 12)
(15347, 12)
(15347, 12)
(15347, 12)
(15347, 12)
(15347, 12)
